<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="ITM">
</head>

<img src="https://maua.br/images/selo-60-anos-maua.svg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->



# **P1 - RA PAR**

In [ ]:
#@markdown Nome completo e RA (*\<nome\>, \<RA\>*)
Aluno = 'Carlos Freire Maia de Oliveira, 21007810' #@param {type:"string"}






In [1]:
path = 'https://github.com/Rogerio-mack/work/raw/main/data/'
import pandas as pd
import numpy as np

# **Case**: Airbnb

Neste case, analise dados sobre imóveis alugados pela plataforma Airbnb em NYC no ano de 2019.


In [24]:
df = pd.read_csv(path + 'AB_NYC_2019.csv')
df.head()

id                                            name  host_id  \
0   20913              Charming 1 bed GR8 WBurg LOCATION!    79402   
1   25235               Large 2 Bedroom Great for Groups!    87773   
2   76761  Eveland the Place to Stay & Enjoy a 5-⭐️ 2bdrm   118971   
3  135465             Garden apartment close to Manhattan   663879   
4  169002                Modern Space in Charming Pre-war   805344   

     host_name neighbourhood_group       neighbourhood  latitude  longitude  \
0   Christiana            Brooklyn        Williamsburg  40.70984  -73.95775   
1       Shelly            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
2       Evelyn            Brooklyn         South Slope  40.66552  -73.99019   
3  Christopher            Brooklyn         Fort Greene  40.68626  -73.97598   
4         Alec           Manhattan              Harlem  40.82411  -73.94934   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0  Entire home/apt    100               5                168  2018-07-22   
1  Entire home/apt    125              90                162  2019-06-28   
2  Entire home/apt    169               2                398  2019-06-28   
3  Entire home/apt    170               3                 86  2019-06-20   
4     Private room     65               2                 41  2019-06-16   

   reviews_per_month  calculated_host_listings_count  availability_365  
0               1.57                               1                 0  
1               1.46                               2               137  
2               3.97                               3               182  
3               0.91                               2               286  
4               0.43                               2                59

# Q1. Proprietários com mais imóveis na plataforma

Obtenha os 3 proprietários ("hosts") com mais imóveis sendo anunciados na plataforma, e a respectiva quantidade de imóveis anunciados.


In [22]:
top_hosts = df['host_id'].value_counts().head(3).reset_index()
top_hosts.columns = ['host_id', 'listings_count']

top_hosts = top_hosts.merge(df[['host_id', 'host_name']].drop_duplicates(), on='host_id', how='left')

display(top_hosts)

host_id  listings_count     host_name
0  219517861               7  Sonder (NYC)
1  120762452               4       Stanley
2   30283594               4          Kara

# Q2A. Locatários profissionais $\times$ anunciantes com um único imóvel

Considere o preço médio dos imóveis. Qual a diferença do preço médio dos imóveis dos anunciantes com mais de um imóvel e dos anunciantes com um único imóvel anunciado na plataforma?

In [4]:
host_counts = df['host_id'].value_counts()
df['host_listings'] = df['host_id'].map(host_counts)

mais_de_um = df[df['host_listings'] > 1]
apenas_um = df[df['host_listings'] == 1]

preco_mais_de_um_imovel = mais_de_um['price'].mean()
preco_mais_um_imovel = apenas_um['price'].mean()

print( f'Diferença de preços: USD {preco_mais_de_um_imovel - preco_mais_um_imovel:.2f}')

Diferença de preços: USD 67.81


# Q3A.  Manhattan *outliers*

Qual o percentual de imóveis de Manhattan com preços estatisticamente discrepantes (*outliers*) com relação aos preços de todos os imóveis?



In [5]:
q1 = df['price'].quantile(0.25)
q3 = df['price'].quantile(0.75)
iqr = q3 - q1

limite_inferior = q1 - 1.5 * iqr
limite_superior = q3 + 1.5 * iqr

df_manhattan = df[df['neighbourhood_group'] == 'Manhattan']

outliers_manhattan = df_manhattan[
    (df_manhattan['price'] < limite_inferior) | (df_manhattan['price'] > limite_superior)
]

perc_Manhattan = (len(outliers_manhattan) / len(df_manhattan)) * 100

print(f'Percentuais de outliers de preços, Manhattan: {perc_Manhattan:.2f} %')

Percentuais de outliers de preços, Manhattan: 10.05 %


# Q4A. Preços $\times$ Descrições

Considere as descrições dos anúncios em `df.name`. Crie uma coluna `df['word']` que contém 'beautiful','sunny','spacious' ou 'heart' de acordo com a presença dessa palavra no anúncio. Caso não tenha nenhuma das palavras, assinale 'others'. Ao final, verifique a quantidade e o preço médio dos imóveis para cada grupo dessas palavras nos anúncios.

Atenção, considere a ordem 'beautiful','sunny','spacious' ou 'heart' (isto é, se o texto contém duas ou mais palavras, por exemplo 'beautiful' e 'sunny', considere 'beautiful'). Considere também quaisquer combinações de maiúsculas/minúsculas nas palavras.

In [6]:
import pandas as pd

def categorize_description(name):
    name = str(name).lower()
    if 'beautiful' in name:
        return 'beautiful'
    elif 'sunny' in name:
        return 'sunny'
    elif 'spacious' in name:
        return 'spacious'
    elif 'heart' in name:
        return 'heart'
    else:
        return 'others'

df['word'] = df['name'].apply(categorize_description)

resultado = df.groupby('word').agg(
    quantidade=('name', 'count'),
    preco_medio=('price', 'mean')
).reset_index()

resultado

word  quantidade  preco_medio
0  beautiful          41   137.926829
1      heart          36   166.388889
2     others         623   157.886218
3   spacious          60   128.516667
4      sunny          43   137.674419

# Q5. Enriquecendo os dados

Enriqueça os dados dos imóveis com os valores dos impostos e multas municipais. Cacule a média de impostos e multas de cada grupo de vizinhaça, `neighbourhood_group`. Note que existem imóveis sem impostos e sem multas, esses são imóveis isentos, mas devem ser considerados nos cálculos de média (por exemplo, você pode considerar esses imóveis com valor 0).

In [7]:
df_taxes = pd.read_csv(path + 'AB_NYC_2019_taxes.csv')
df_taxes

id   type   value
0     35907127  taxes  1315.0
1       804815  taxes     NaN
2     22328585  taxes   900.0
3     27876600  taxes   869.0
4     34976037  taxes  1629.0
...        ...    ...     ...
1523  34284316  fines     NaN
1524  20325804  fines     NaN
1525  25872986  fines     NaN
1526  19365218  fines   906.0
1527   4235843  fines     NaN

[1528 rows x 3 columns]

In [8]:
df_taxes_tax = df_taxes[df_taxes['type'] == 'taxes'][['id', 'value']].rename(columns={'value': 'taxes'})
df_taxes_fine = df_taxes[df_taxes['type'] == 'fines'][['id', 'value']].rename(columns={'value': 'fines'})

df = df.merge(df_taxes_tax, on='id', how='left')
df = df.merge(df_taxes_fine, on='id', how='left')

df[['taxes', 'fines']] = df[['taxes', 'fines']].fillna(0)

df_taxes_e_fines = df.groupby('neighbourhood_group')[['taxes', 'fines']].mean().reset_index()

display(df_taxes_e_fines)

neighbourhood_group       taxes       fines
0               Bronx  808.500000  120.150000
1            Brooklyn  798.457576  292.554545
2           Manhattan  855.029891  302.997283
3              Queens  793.987342  318.683544
4       Staten Island  992.142857  498.285714

# Q6. Quebras

Construa no `Pandas` um "relatório" (`df_report`) como abaixo. Os valores são ordenados `neighbourhood_group`,`neighbourhood`. Cada "página" contém apenas 30 linhas. A cada "quebra" de `neighbourhood_group` é gerada uma linha com preço médio do `neighbourhood_group` e, então, é iniciada uma nova página.

O item `line_id` é opcional mas deve auxiliá-lo nessa tarefa.

Ao final produza a soma dos preços das páginas abaixo...

In [13]:
df_sorted = df.sort_values(['neighbourhood_group', 'neighbourhood'])

report_lines = []
page_id = 1
line_id = 0

for group, group_data in df_sorted.groupby('neighbourhood_group'):
    linhas_grupo = []

    for _, row in group_data.iterrows():
        if line_id == 30:
            page_id += 1
            line_id = 0
        linhas_grupo.append({
            'neighbourhood_group': group,
            'neighbourhood': row['neighbourhood'],
            'price': row['price'],
            'page_id': page_id,
            'line_id': line_id
        })
        line_id += 1

    if line_id == 30:
        page_id += 1
        line_id = 0
    mean_price = group_data['price'].mean()
    linhas_grupo.append({
        'neighbourhood_group': group,
        'neighbourhood': 'MEAN',
        'price': mean_price,
        'page_id': page_id,
        'line_id': line_id
    })
    line_id += 1

    page_id += 1
    line_id = 0

    report_lines.extend(linhas_grupo)

df_report = pd.DataFrame(report_lines)

sum_price_pages = df_report[df_report.page_id.isin([1, 24, 25, 28])].groupby('page_id')['price'].sum().astype(int).sum()

print(f'A soma dos preços das páginas 1, 24, 25 e 28 é: {sum_price_pages}')

A soma dos preços das páginas 1, 24, 25 e 28 é: 12633


#### Aqui um *sample* do report.

In [14]:
df_report.iloc[-10::,:]

neighbourhood_group  neighbourhood       price  page_id  line_id
799              Queens       Woodside  120.000000       29       18
800              Queens           MEAN   92.227848       29       19
801       Staten Island  Arden Heights   83.000000       30        0
802       Staten Island  Randall Manor   99.000000       30        1
803       Staten Island    South Beach   20.000000       30        2
804       Staten Island    South Beach  100.000000       30        3
805       Staten Island     St. George  130.000000       30        4
806       Staten Island      Todt Hill  135.000000       30        5
807       Staten Island  Tompkinsville   59.000000       30        6
808       Staten Island           MEAN   89.428571       30        7

Q7. Considere as fases do processo CRISP DM. Relacione ao menos 6 tarefas presentes na preparação dos dados.

Todas as 6 tarefas a seguir envolvem a preparação dos dados para resolução de problemas e/ou análises.

Seleção dos dados:
Escolher quais colunas são relevantes para o problema.

Limpeza dos dados:
Corrigir ou remover dados incorretos ou ausentes.

Criação de variáveis:
Criar novas variáveis relevantes a partir das existentes.

Integração de dados:
Combinar diferentes fontes de dados em um único conjunto.

Formatação dos dados:
Transformar os dados para formatos mais apropriados para análise.

Detecção e tratamento de outliers:
Identificar valores muito fora do comum que podem distorcer as análises.